DATA2001 Group Assignment <br>
Task 1: Import and Clean <br>
Package Imports 

In [1]:
# package load
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.wkt import loads
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
SA2 = gpd.read_file('data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
future = gpd.read_file('data/catchments/catchments_future.shp')
primary = gpd.read_file('data/catchments/catchments_primary.shp')
secondary = gpd.read_file('data/catchments/catchments_secondary.shp')
sa2 = SA2

Load data

In [3]:
businesses = pd.read_csv('data/Businesses.csv')
stops = pd.read_csv('data/Stops.txt')
polls = pd.read_csv('data/PollingPlaces2019.csv')
pops = pd.read_csv('data/Population.csv')
income = pd.read_csv('data/Income.csv')

# json file - sydney enrolment + school locations
enrol = pd.read_json('data/enrolment.json')


SQL Queries

In [4]:
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json
import pandas as pd

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [5]:
db, conn = pgconnect(credentials)

Connected successfully.


In [6]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [7]:
srid = 4283

# drop row if geometry is None
# SA2
sa2 = sa2[sa2.geometry.notnull()]
# polls
polls = polls[polls.the_geom.notnull()]
# enrolment
enrol = enrol.dropna(subset=['Latitude'])
enrol = enrol.dropna(subset=['Longitude'])

# convert geometry to WKTElement
# polls
polls['geom'] = gpd.points_from_xy(polls.longitude, polls.latitude)
polls['geom'] = polls['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

# enrolment
enrol['geom'] = gpd.points_from_xy(enrol.Longitude, enrol.Latitude)
enrol['geom'] = enrol['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

# convert polygon to multipolygon
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid=srid)

# convert geometry to WKTElement
sa2['geom'] = sa2['geometry'].apply(lambda x: create_wkt_element(x, srid))

# drop old geometry column
polls = polls.drop('the_geom', axis=1)
sa2 = sa2.drop('geometry', axis=1)

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
# subset sa2
sa2['sa2_code21'] = sa2['SA2_CODE21']
sa2['sa2_name21'] = sa2['SA2_NAME21']
sa2 = sa2[['sa2_code21', 
          'sa2_name21',
          'geom']]

In [9]:
# subset polling
polls = polls[['polling_place_name',
               'premises_name',
               'geom']].reset_index(drop=True)

In [10]:
# subset enrolment
enrol = enrol[['School_code',
               'School_name',
               'latest_year_enrolment_FTE',
               'geom']]

# change all variables to lowercase
enrol = enrol.rename(columns={'School_code': 'school_code',
                              'School_name': 'school_name',
                              'latest_year_enrolment_FTE': 'latest_year_enrolment_fte'})

# change variable types
enrol['school_code'] = enrol['school_code'].astype(int)
enrol['school_name'] = enrol['school_name'].astype(str)

# drop any value that is not a number from latest_year_enrolment_fte
enrol = enrol[enrol['latest_year_enrolment_fte'].str.strip().replace('', pd.NA).notna()]
enrol['latest_year_enrolment_fte'] = enrol['latest_year_enrolment_fte'].astype(float)

# drop missing values
enrol = enrol.dropna(subset=['school_code'])
enrol = enrol.dropna(subset=['school_name'])
enrol = enrol.dropna(subset=['latest_year_enrolment_fte'])
enrol = enrol.reset_index(drop=True)


In [11]:
enrol

,school_code,school_name,latest_year_enrolment_fte,geom
0,8473,Chifley College Senior Campus,471.0,POINT (150.829855 -33.769305)
1,8325,Moree Secondary College,389.0,POINT (149.842344 -29.46645)
2,8870,St Marys Senior High School,874.4,POINT (150.770024 -33.763119)
3,8374,Brisbane Water Secondary College Woy Woy Campus,663.4,POINT (151.323582 -33.492687)
4,8539,Sydney Secondary College Blackwattle Bay Campus,777.2,POINT (151.188357 -33.875142)
...,...,...,...,...
2165,4678,Ngarala Public School,168.0,POINT (151.069192 -33.78106)
2166,4682,Gregory Hills Public School,233.0,POINT (150.78 -34.025)
2167,4683,Murrumbateman Public School,107.0,POINT (149.030288 -34.9664751)
2168,4690,Jindabyne Public School,285.0,POINT (148.618556 -36.416783)


In [12]:
# create schema SA2
sql = """
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    sa2_code21 VARCHAR(20),
    sa2_name21 VARCHAR(150),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [13]:
# create schema polling
sql = """
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    polling_place_name VARCHAR(50),
    premises_name VARCHAR,
    latitude FLOAT,
    longitude FLOAT,
    geom GEOMETRY(POINT, 4283)
);
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [14]:
# create schema enrolment
# create schema enrol
sql = """
DROP TABLE IF EXISTS enrol;
CREATE TABLE enrol (
    school_code INTEGER,
    school_name VARCHAR,
    latest_year_enrolment_fte FLOAT,
    geom GEOMETRY(Point, 4283)
);
"""

query(conn, sql)


Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [15]:
# SA2 input
sa2.to_sql('sa2', conn, if_exists='append', index=False, 
           dtype={'geom': Geometry('MULTIPOLYGON', srid)})

# polls nput 
polls.to_sql('polls', conn, if_exists='append', index=False, 
             dtype={'geom': Geometry('POINT', srid)})

# enrolment input
enrol.to_sql('enrol', conn, if_exists='append', index=False, 
             dtype={'geom': Geometry('POINT', srid)})

170

In [16]:
sql = """
select * from enrol
"""

query(conn, sql)

,school_code,school_name,latest_year_enrolment_fte,geom
0,8473,Chifley College Senior Campus,471.0,0101000020BB10000082AD122C8EDA62401CD3139678E2...
1,8325,Moree Secondary College,389.0,0101000020BB1000006B7F677BF4BA62401D3867446977...
2,8870,St Marys Senior High School,874.4,0101000020BB10000053245F09A4D8624066FA25E2ADE1...
3,8374,Brisbane Water Secondary College Woy Woy Campus,663.4,0101000020BB1000006072A3C85AEA624009151C5E10BF...
4,8539,Sydney Secondary College Blackwattle Bay Campus,777.2,0101000020BB100000205F420507E6624092AD2EA704F0...
...,...,...,...,...
2165,4678,Ngarala Public School,168.0,0101000020BB100000A22424D236E262405C1B2AC6F9E3...
2166,4682,Gregory Hills Public School,233.0,0101000020BB100000295C8FC2F5D86240333333333303...
2167,4683,Murrumbateman Public School,107.0,0101000020BB100000C32E8A1EF8A06240FC72C174B57B...
2168,4690,Jindabyne Public School,285.0,0101000020BB100000D4D7F335CB936240AF4335255935...


In [17]:
sql = """
select sa2.sa2_code21, 
sa2.sa2_name21, 
count(enrol.school_name) as school_count, 
sum(enrol.latest_year_enrolment_fte) as total_enrolment
from sa2 
left join enrol on ST_Intersects(enrol.geom, sa2.geom)
group by sa2.sa2_code21, sa2.sa2_name21
order by school_count desc;
"""

query(conn, sql)

,sa2_code21,sa2_name21,school_count,total_enrolment
0,112021249,Lismore Surrounds,17,999.2
1,113031271,Wagga Wagga Surrounds,13,967.2
2,110041205,Tamworth Surrounds,13,1043.2
3,103041079,Orange Surrounds,12,1123.0
4,113011257,Griffith Surrounds,12,1096.0
...,...,...,...,...
2449,302011026,McDowall,0,NaN
2450,302021027,Aspley,0,NaN
2451,302021028,Chermside,0,NaN
2452,302021029,Chermside West,0,NaN


In [18]:
# calculate z scores 
sql = """
WITH counts AS (
    SELECT 
        sa2.sa2_code21, 
        sa2.sa2_name21, 
        COUNT(enrol.school_name) AS school_count, 
        SUM(enrol.latest_year_enrolment_fte) AS total_enrolment
    FROM 
        sa2 
    LEFT JOIN 
        enrol ON ST_Intersects(enrol.geom, sa2.geom)
    GROUP BY 
        sa2.sa2_code21, sa2.sa2_name21
),
non_zero_counts AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        school_count,
        total_enrolment
    FROM 
        counts
    WHERE 
        school_count > 0 AND total_enrolment > 0
),
z_scores AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        school_count,
        total_enrolment,
        (school_count - AVG(school_count) OVER ()) / NULLIF(STDDEV(school_count) OVER (), 0) AS school_count_zscore,
        (total_enrolment - AVG(total_enrolment) OVER ()) / NULLIF(STDDEV(total_enrolment) OVER (), 0) AS total_enrolment_zscore
    FROM 
        non_zero_counts
)
SELECT 
    sa2_code21, 
    sa2_name21, 
    school_count_zscore,
    total_enrolment_zscore
FROM 
    z_scores
ORDER BY 
    school_count_zscore DESC;
"""

query(conn, sql)

,sa2_code21,sa2_name21,school_count_zscore,total_enrolment_zscore
0,112021249,Lismore Surrounds,5.840034,-0.344630
1,113031271,Wagga Wagga Surrounds,4.092990,-0.379844
2,110041205,Tamworth Surrounds,4.092990,-0.296212
3,101051540,Goulburn Surrounds,3.656229,-0.679820
4,113011257,Griffith Surrounds,3.656229,-0.238109
...,...,...,...,...
593,119021660,Campsie - South,-1.148141,-0.786121
594,124041468,Yarramundi - Londonderry,-1.148141,-1.215287
595,102011038,Saratoga - Davistown,-1.148141,-1.019411
596,119011655,Greenacre - North,-1.148141,-1.010607


# Analysis

## Numerical Analysis

In [ ]:
# read in sigmoid function
sig = pd.read_csv('sydney_demographics/fullquery_sig.csv')

In [48]:
# max  sigmoid score
max_sig = sig[sig['final_score_zsum'] == sig['final_score_zsum'].max()]
min_sig = sig[sig['final_score_zsum'] == sig['final_score_zsum'].min()]
median_sig = sig[sig['final_score_zsum'] == sig['final_score_zsum'].median()]
mean_sig = sig.final_score_zsum.mean()

,sa2_code21,sa2_name21,stop_count_zscore,school_area_1000_zscore,polling_count_zscore,total_businesses_zscore,total_median_zscore,school_enrollment_count_zscore,incidents_count_zscore,final_score_zsum,distance_to_cbd_km,east_west
183,119041382,Sans Souci - Ramsgate,0.273544,-0.117748,-0.016912,0.315966,0.0,-0.489561,-0.058122,0.476808,13.129215,West


The 'bustling' metric was created using z-scores from the provided datasets, including polling locations, bus stops, school area sizes, and total businesss. Additional data was also collected, including total student enrolment within the SA2 area. The mean sigmoid function score was 0.483, with the median 'bustling' SA2 area being Sans Souci, with a score of 0.477. The most 'bustling' area of Sydney was determined to be Badgerys Creek, with a sigmoid function socre of ~1. The least 'bustling' area was determined to be Cobbity, with a sigmoid function score of ~0.006. 

## Distance & Bustlingness

The sigmoid function can be utilised in order to demonstrate different trends of 'bustlingness' throughout Sydney's GMR. One interesting trend that can be studied is the relationship between 'bustlingness' and the relative position in relation to the CBD. A simple linear regression model was conducted in order to test for any significant relationships between the two variables. 

In [20]:
# calculate distance to sydney cbd 
sql = """
WITH cbd AS (
    SELECT 
        ST_SetSRID(ST_MakePoint(151.2073, -33.8708), 4283) AS geom
),
sa2_cbd AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        ST_Distance(sa2.geom::geography, cbd.geom::geography) AS distance_to_cbd
    FROM 
        sa2 
    CROSS JOIN 
        cbd
)
SELECT
    sa2_code21,
    sa2_name21,
    distance_to_cbd / 1000 AS distance_to_cbd_km
FROM    
    sa2_cbd
ORDER BY    
    distance_to_cbd_km;
"""

distance = query(conn, sql)
distance.to_csv('distance_to_cbd.csv', index=False)

In [21]:
# read in distances 
dist = pd.read_csv('distance_to_cbd.csv')

# add distances to sig data
sig = sig.merge(dist, on=['sa2_code21', 'sa2_name21'], how='left')

In [22]:
# regression of final_score_zsum ~ distance_to_cbd_km in python
import statsmodels.api as sm

# drop missing values and 0s
sig = sig.dropna(subset=['distance_to_cbd_km'])
sig = sig[sig['distance_to_cbd_km'] > 0]
sig = sig.dropna(subset=['final_score_zsum'])
sig = sig[sig['final_score_zsum'] > 0]

# regression
X = sig['distance_to_cbd_km']
y = sig['final_score_zsum']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       final_score_zsum   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.449
Date:                Tue, 14 May 2024   Prob (F-statistic):              0.118
Time:                        01:29:08   Log-Likelihood:                -135.60
No. Observations:                 367   AIC:                             275.2
Df Residuals:                     365   BIC:                             283.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.5215      0.031     17.080      0.000       0.461       0.582
distance_to_cbd_km    -0.0016      0.001     -1.565      0.118      -0.004       0.000
==============================================================================
Omnibus:                     3366.750   Durbin-Watson:                   0.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.657
Skew:                           0.136   Prob(JB):                     1.10e-08
Kurtosis:                       1.476   Cond. No.                         48.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The regression model, run using python's `statsmodels` module, showed no signficant relationship between the 'bustling' metric and distance from the CBD (t = -1.565, p = 0.118, df = 365). The interactive leaflet already suggested that this might be the case as SA2 areas with high 'bustling' metric scores can be observed across Sydney's entire GMR. Further, low R-squared and adjusted R-squared values (0.007 and 0.004 respectively) suggest that the model does not explain much of the variance in the data.

In [23]:
# plot using plotly express
import plotly.express as px

fig = px.scatter(sig, x='distance_to_cbd_km', y='final_score_zsum', trendline='ols')
fig.update_traces(marker=dict(color='blue'))
fig.update_traces(line=dict(color='darkblue'))
fig.update_xaxes(title_text='Distance to Sydney CBD (km)')
fig.update_yaxes(title_text='Sigmoid Function Score')
fig.update_layout(title='Distance to Sydney CBD vs Sigmoid Function Score')
fig.update_traces(hovertemplate='Distance to CBD: %{x:.3f} km<br>Sigmoid Function Score: %{y:.3f}')
fig.show()



The scatterplot created using `plotly.express` visualises the trend. A slight negative relationship can be observed, however, the relationship is not significant. Therefore, the 'bustling' metric is not dependent on the distance from the CBD.

## Comparison of East and West Sydney

Given that both economic and social factors were utilised in the creation of the 'bustling' metric, it would be interesting to compare the scores outputed by the sigmoid function for East and West Sydney. A simple t-test assuming equal variance can be conducted in order to evaluate whether there is a significant difference between the two groups.  

In [24]:
# create east_west variable
sql = """
WITH east_west AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        CASE 
            WHEN ST_XMax(geom) > 151.1511 THEN 'East' 
            ELSE 'West' 
        END AS east_west
    FROM 
        sa2
)
SELECT
    sa2_code21,
    sa2_name21,
    east_west
FROM
    east_west;
"""

east_west = query(conn, sql)
east_west.to_csv('east_west.csv', index=False)

In [25]:
# merge east_west with sig
east_west = pd.read_csv('east_west.csv')
sig = sig.merge(east_west, on=['sa2_code21', 'sa2_name21'], how='left')

In [28]:
# perform t-test sigmoid ~ east_west
from scipy import stats

east = sig[sig['east_west'] == 'East']
west = sig[sig['east_west'] == 'West']

ttest = stats.ttest_ind(east['final_score_zsum'], west['final_score_zsum'])
ttest

TtestResult(statistic=-0.02013804476529405, pvalue=0.9839442547718726, df=365.0)

The two-sample t-test revelaed a non-singificant difference in the sigmoid function scores (t = 0.020, p = 0.984, df = 365). This suggests that there is no difference in 'bustlingness' between East and West Sydney. 

In [29]:
# plot using plotly express
fig = px.box(sig, x='east_west', y='final_score_zsum', points='all')
fig.update_traces(marker=dict(color='blue'))
fig.update_traces(line=dict(color='darkblue'))
fig.update_xaxes(title_text='East vs West')
fig.update_yaxes(title_text='Sigmoid Function Score')
fig.update_layout(title='East vs West Sigmoid Function Score')
fig.update_traces(hovertemplate='East/West: %{x}<br>Sigmoid Function Score: %{y:.3f}')
fig.show()

The two boxplots show the distribution of the sigmoid function score for the east and west regions of Sydney. The non-significant result becomes evident when observing the similarity in the distribution of scores between the two groups. 